# Capstone Project
### IBM Data Science Specialization - Course 9
_this notebook will be used for the Capstone Project - Battling Neighbourhoods_

## Importing libraries

In [141]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml

## Web scraping with BeautifulSoup library

In [159]:
#getting the sourse html of the webpage
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#creating the BS object to work with
soup = BeautifulSoup(source, 'lxml')

In [160]:
#fetching the table component from the html wikipedia page
table = soup.find("table") 

#each cell represents a row in the future DataFrame
cells = table.find_all("tr")

#we will make DataFrame from these arrays
PostalCode   = list()
Borough      = list()
Neighborhood = list()

#going through the cells and filling out the arrays
for i in range(1,len(cells)):
    current_elems = cells[i].find_all("td")
    
    nbhood = str(current_elems[2].text) #some issues with the last column 
    
    PostalCode.append(current_elems[0].text)
    Borough.append(current_elems[1].text)
    Neighborhood.append(nbhood[0:len(nbhood)-1])

In [161]:
data_ = {"PostalCode" : PostalCode, "Borough": Borough, "Neighborhood": Neighborhood}
df =pd.DataFrame(data=data_)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [162]:
#Processing the DataFrame according to given conditions

#1: all entries with Borough = Not Assigned must be dropped

df = df.loc[df['Borough'] != 'Not assigned'].reset_index()


#2: If neighborhood is unavailable, we copy borough into it

df.loc[df['Neighborhood'] == "Not assigned", 'Neighborhood'] = df['Borough']

df = df[['PostalCode','Borough','Neighborhood']]

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [163]:
#3: grouping neighborhoods by postal code

print(df['PostalCode'].value_counts().tolist())
print(len(df['PostalCode'].value_counts().tolist()))

[8, 8, 7, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
103


In [164]:
for elem in df['PostalCode'].tolist():
    #print('elem: ' + str(elem))
    borough_arr = df.loc[df['PostalCode'] == elem,'Borough'].tolist()
    #print(borough_arr)
    neighborhoods = df.loc[df['PostalCode']==elem,'Neighborhood'].tolist()
    neighbString  = ""
    
    for i in neighborhoods:
        neighbString += (i+", ")
        
    df = df.loc[df['PostalCode'] != elem]
    df = df.append({'PostalCode': elem, 'Borough': borough, 'Neighborhood': neighbString[0:-2]},ignore_index=True)
    df.reset_index()

In [166]:
df.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Downtown Toronto,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,Downtown Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Downtown Toronto,"Humber Bay, King's Mill Park, Kingsway Park So..."
102,M8Z,Downtown Toronto,"Kingsway Park South West, Mimico NW, The Queen..."


In [167]:
df.shape

(103, 3)